# Unified Data Preprocessing
#### Fire Atlas Dataset
Andrew McDonald // CSE 847 // April 2021

In [1]:
import requests
import urllib
import json
from datetime import datetime

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
ignitions = gpd.read_file("2016_wildfires/Global_fire_atlas_V1_ignitions_2016.shp")
ignitions

,fire_ID,latitude,longitude,size,perimeter,start_date,start_DOY,end_date,end_DOY,duration,expansion,fire_line,speed,direction,direction_s,landcover,landcover_s,tile_ID,geometry
0,1,20.8438,-156.4130,0.86,4.63,2016-10-26,300,2016-11-02,307,8,0.11,0.23,0.37,3,east,12,Croplands,h03v06,POINT (-156.41374 20.84379)
1,2,20.8396,-156.4180,1.50,6.48,2016-10-28,302,2016-11-02,307,6,0.25,0.54,0.57,4,southeast,12,Croplands,h03v06,POINT (-156.41833 20.83962)
2,3,20.8105,-156.4370,0.43,2.78,2016-11-29,334,2016-11-29,334,1,0.43,0.93,0.46,0,none,12,Croplands,h03v06,POINT (-156.43708 20.81045)
3,4,20.6271,-156.2470,18.65,30.56,2016-02-16,47,2016-02-20,51,5,3.73,6.20,2.68,8,northwest,7,Open shrublands,h03v06,POINT (-156.24786 20.62712)
4,5,19.8063,-155.8950,7.72,12.96,2016-03-20,80,2016-03-25,85,6,1.29,2.47,1.39,4,southeast,7,Open shrublands,h03v07,POINT (-155.89512 19.80629)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887216,887217,-15.2604,145.1189,1.50,6.48,2016-11-21,326,2016-11-23,328,3,0.50,1.08,0.91,3,east,8,Woody savannas,h32v10,POINT (145.11887 -15.26036)
887217,887218,-15.2770,145.1347,0.64,3.70,2016-11-17,322,2016-11-21,326,5,0.13,0.28,0.51,5,south,8,Woody savannas,h32v10,POINT (145.13471 -15.27703)
887218,887219,-15.3104,145.1535,2.57,7.41,2016-10-21,295,2016-10-30,304,10,0.26,0.56,0.97,3,east,8,Woody savannas,h32v10,POINT (145.15348 -15.31036)
887219,887220,-15.3020,145.1779,1.50,5.56,2016-10-02,276,2016-10-08,282,7,0.21,0.46,0.75,8,northwest,8,Woody savannas,h32v10,POINT (145.17794 -15.30203)


Excellent. We have over 887,220 fires, plenty to train a deep model upon.
Let's link them up with weather data from the NCDC.
We begin by finding all weather stations available in the [2016 NCDC GSOM
dataset](https://www.ncdc.noaa.gov/cdo-web/webservices/v2) that have the variables we hope to use as predictors.

In [ ]:
api_token = "syKpWfFaNGhSsksXrJhcrwQspIYihUqW"
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/"
headers = {"token": api_token}
resource = "stations?"
query = {
    "datasetid": "GSOM",
    "startdate": "2016-01-01",
    "enddate": "2016-01-01",
    "limit": "1000"
}
query_str = urllib.parse.urlencode(query, doseq=False)
endpoint = base_url + resource + query_str
response = requests.get(endpoint, headers=headers)
results = json.loads(response.text)
results["metadata"]["resultset"]